```bash
%%bash
hdfs dfs -rmr wordcount/output2 2>/dev/null
comparator_class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator
mapred streaming \
  -D mapreduce.job.output.key.comparator.class=$comparator_class \
  -D mapreduce.partition.keycomparator.options=-nr \
  -file swap_keyval.sh \
  -input wordcount/output \
  -output wordcount/output2 \
  -mapper swap_keyval.sh
```


### Что такое Hadoop Streaming <a name="hadoopstreaming"></a>

Hadoop Streaming - это библиотека в Hadoop, которая разработка для созданиях самописных мапперов и редьюсеров в исполняемые процессы MapReduce. 


Mapper и reducer читают данные из stdin и отправляют их в stdout. Обучно, колонки в данных разделяются с помощью `tab`. Если данные разделены другим разделителем, то надо будет определять разделитель. Для этого ознакомьтесь с `TextInputFormat` (see the [API documentation](https://hadoop.apache.org/docs/stable/api/org/apache/hadoop/mapred/TextInputFormat.html)) и [Hadoop Streaming documentation](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html#Customizing_How_Lines_are_Split_into_KeyValue_Pairs).

Пример MapReduce streaming синтаксиса:
<html>
<pre>
    mapred streaming \
  -input myInputDirs \
  -output myOutputDir \
  -mapper /bin/cat \
  -reducer /usr/bin/wc

</pre>
</html>

Документация для Hadoop Streaming от Apache Hadoop: [https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html).

Все настройки Hadoop Streaming и опции описаны здесь: [Streaming Command Options](https://hadoop.apache.org/docs/current/hadoop-streaming/HadoopStreaming.html#Streaming_Command_Options)

# MapReduce на mrjob



mrjob это библиотека для Python, которая позволяет создавать MapReduce jobs. 

Документация по MRJob: [https://mrjob.readthedocs.io/en/latest/](https://mrjob.readthedocs.io/en/latest/)

Возьмем файл из MapReduce(bash) `fruits.txt`

In [6]:
%%bash
ls -lh fruits.txt

-r--r--r-- 1 user123 hadoopusers 429M Apr 17 23:00 file.txt


Создадим файл `word_count.py` с помощью magic в Jupyter Notebook `%%file`

In [10]:
%%file word_count.py
from mrjob.job import MRJob

class MRWordFrequencyCount(MRJob):
    
    def mapper(self, _, line):
        
        yield "chars", len(line)
        yield "words", len(line.split())
        yield "lines", 1


    def reducer(self, key, values):
        yield key, sum(values)


if __name__ == '__main__':
    MRWordFrequencyCount.run()

Overwriting word_count.py


Мы можем определит кол-во мапперов и редьюсеров, на пример $10$ мапперов и $3$ редьюсеров. (игрушечный пример)

In [ ]:
%%bash

DATAFILE=fruits.txt
STREAMING_JAR=/opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar
N=10

# N map tasks
python word_count.py\
    --jobconf mapreduce.job.maps=$N\
    --jobconf mapreduce.job.reduces=3\
    -r hadoop\
    --hadoop-streaming-jar $STREAMING_JAR\
    $DATAFILE

Проведем эксперименты

In [ ]:
%%bash
START=$(date +%s);

DATAFILE=# файл с большим набором данных
STREAMING_JAR=/opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar
N=4 # кол-во мапперов

# N map tasks
python word_count.py\
    --jobconf mapreduce.job.maps=$N\
    --jobconf mapreduce.job.reduces=3\
    -r hadoop --hadoop-streaming-jar $STREAMING_JAR\
    $DATAFILE
    
2>/dev/null

END=$(date +%s);
echo $((END-START)) | awk '{print "Duration: "int($1/60)":"int($1%60)}'